This notebook is provided by [Ki Yung Ahn](https://kyagrd.github.io/) as a course material for the compiler course at
[Dept. of Computer Engineering](http://ce.hannam.ac.kr), [Hannam University](http://www.hnu.kr/), Daejeon, Korea.

In [2]:
type Nm = String

type State = Int
type Sigma = Char
data Gamma = N Nm     -- Non-terminal symbol
           | T Sigma  -- Terminal symbol
           | Z        -- initial stack symbol
           deriving (Eq, Show)
type Delta = [(State, Maybe Sigma, Maybe Gamma, [Gamma], State)]

-- We assume that the set of states are consecutive integers
-- from the inital state (min state) to the max state, inclusively.
-- (max state, transitions, initial state, final states)
type PDA = (State, Delta, State, [State])

In [3]:
delta :: PDA -> State -> Maybe Sigma -> Maybe Gamma -> [(State, [Gamma])]
delta (_, d, _, _) p ms mg =
  [(q,gs) | (p',ms',mg',gs,q)<-d, p==p', ms==ms', mg==mg']

In [4]:
step pda (p,   [],  [])   =
  [(q,   [], gs')        | (q,gs') <- delta pda p Nothing  Nothing ]
step pda (p, c:cs,   [])  =
  [(q, c:cs, gs')        | (q,gs') <- delta pda p Nothing  Nothing ] ++
  [(q,   cs, gs')        | (q,gs') <- delta pda p (Just c) Nothing ]
step pda (p,   [], g:gs) =
  [(q,   [], gs'++ g:gs) | (q,gs') <- delta pda p Nothing  Nothing ] ++
  [(q,   [], gs'++   gs) | (q,gs') <- delta pda p Nothing  (Just g)]
step pda (p, c:cs, g:gs) =
  [(q, c:cs, gs'++ g:gs) | (q,gs') <- delta pda p Nothing  Nothing ] ++
  [(q,   cs, gs'++ g:gs) | (q,gs') <- delta pda p (Just c) Nothing ] ++
  [(q, c:cs, gs'++   gs) | (q,gs') <- delta pda p Nothing  (Just g)] ++
  [(q,   cs, gs'++   gs) | (q,gs') <- delta pda p (Just c) (Just g)]

:type step

step :: PDA -> (State, [Sigma], [Gamma]) -> [(State, [Sigma], [Gamma])]

In [5]:
(p,cs,gs) |- (q,cs',gs') = \pda -> (q,cs',gs') `elem` step pda (p,cs,gs)

:type (|-)

(|-) :: (State, [Sigma], [Gamma]) -> (State, [Sigma], [Gamma]) -> PDA -> Bool

In [6]:
-- reflexive transitive relation over |- limited by maximum number of steps 
(|-*) :: (State, [Sigma], [Gamma]) -> (State, [Sigma], [Gamma]) -> (PDA, Int) -> Bool
(|-*) (p,cs,gs) (q,cs',gs') (_,   n) | n <= 0 = (p,cs,gs)==(q,cs',gs')
(|-*) (p,cs,gs) (q,cs',gs') (_,   _) | (p,cs,gs)==(q,cs',gs') = True
(|-*) (p,cs,gs) (q,cs',gs') (pda, n) =
  or [(p1,cs1,gs1) |-* (q,cs',gs') $ (pda, n-1) | (p1,cs1,gs1) <- step pda (p,cs,gs)]

PDA for the conext-free language $\displaystyle \{a^nb^n \mid n\ge 0\}$.

* $S \to AZ$
* $A \to \epsilon$
* $A \to aAb$

<img src="https://graphviz.glitch.me/graphviz?layout=dot&format=svg&mode=download&graph=digraph%20G%20%7B%0A%20%20%20%20node%20%5Bshape%3D%22circle%22%5D%3B%0A%09start%20-%3E%200%3B%0A%20%20%20%200%20-%3E%200%20%5Blabel%3D%22%CE%B5%3B%20Z%2FAZ%20%20%5Cn%CE%B5%3B%20A%2FaAb%5Cna%3B%20a%2F%CE%B5%20%20%20%20%20%20%22%5D%3B%0A%090%20-%3E%201%20%5Blabel%3D%22%CE%B5%3B%20A%2F%CE%B5%22%5D%3B%0A%20%20%20%201%20-%3E%201%20%5Blabel%3D%22b%3B%20b%2F%CE%B5%22%5D%3B%0A%091%20-%3E%202%20%5Blabel%3D%22%CE%B5%3B%20Z%2F%CE%B5%22%5D%3B%0A%20%20%20%20%0A%09start%20%5Bshape%3Dnone%5D%3B%0A%20%20%20%202%20%5Bshape%3Ddoublecircle%5D%3B%0A%20%20%20%20%0A%20%20%20%20%2F%2F%20%7Brank%20%3D%20same%3B%20start%3B%200%3B%201%3B%202%3B%7D%0A%7D" />

In [7]:
anbnPDA :: PDA
anbnPDA = (2,d,0,[2])
  where
    d = [ (0, e,__Z, [_A, Z],    0) -- start
        , (0, e,__A, [_a,_A,_b], 0) -- production rule A -> aAb
        , (0, a,__a, [],         0) -- matching input a with stack terminal symbol
        , (0, e,__A, [],         1) -- production rule A -> \epsilon
        , (1, b,__b, [],         1) -- matching input b with stack terminal symbol
        , (1, e,__Z, [],         2) -- finish
        ]
    e = Nothing
    a = Just 'a'
    b = Just 'b'
    __Z = Just Z
    __A = Just _A
    __a = Just _a
    __b = Just _b

_A = N "A"
_a  = T 'a'
_b  = T 'b'

In [8]:
(0,"aabb",            [ Z]) |- (0,"aabb",         [_A, Z]) $ anbnPDA
(0,"aabb",         [_A, Z]) |- (0,"aabb",   [_a,_A,_b, Z]) $ anbnPDA
(0,"aabb",   [_a,_A,_b, Z]) |- (0, "abb",      [_A,_b, Z]) $ anbnPDA
(0, "abb",      [_A,_b, Z]) |- (0, "abb",[_a,_A,_b,_b, Z]) $ anbnPDA
(0, "abb",[_a,_A,_b,_b, Z]) |- (0,  "bb",   [_A,_b,_b, Z]) $ anbnPDA
(0,  "bb",   [_A,_b,_b, Z]) |- (1,  "bb",      [_b,_b, Z]) $ anbnPDA
(1,  "bb",      [_b,_b, Z]) |- (1,   "b",         [_b, Z]) $ anbnPDA
(1,   "b",         [_b, Z]) |- (1,    "",            [ Z]) $ anbnPDA
(1,    "",            [ Z]) |- (2,    "",              []) $ anbnPDA

True

True

True

True

True

True

True

True

True

In [9]:
(0,"aabb",[Z]) |-* (2,"",[]) $ (anbnPDA, 0)
(0,"aabb",[Z]) |-* (2,"",[]) $ (anbnPDA, 1)
(0,"aabb",[Z]) |-* (2,"",[]) $ (anbnPDA, 2)
(0,"aabb",[Z]) |-* (2,"",[]) $ (anbnPDA, 3)
(0,"aabb",[Z]) |-* (2,"",[]) $ (anbnPDA, 4)
(0,"aabb",[Z]) |-* (2,"",[]) $ (anbnPDA, 5)
(0,"aabb",[Z]) |-* (2,"",[]) $ (anbnPDA, 6)
(0,"aabb",[Z]) |-* (2,"",[]) $ (anbnPDA, 7)
(0,"aabb",[Z]) |-* (2,"",[]) $ (anbnPDA, 8)
(0,"aabb",[Z]) |-* (2,"",[]) $ (anbnPDA, 9)
(0,"aabb",[Z]) |-* (2,"",[]) $ (anbnPDA,10)
(0,"aabb",[Z]) |-* (2,"",[]) $ (anbnPDA,11)

False

False

False

False

False

...

False

False

True

True

True

In [10]:
accepts cs pda@(_,_,q0,fs) n = or [(q0,cs,[Z]) |-* (f,"",[]) $ (pda, n) | f<-fs]

In [11]:
accepts "" anbnPDA 10
accepts "ab" anbnPDA 10
accepts "aabb" anbnPDA 10

True

True

True

In [12]:
accepts "a" anbnPDA 10
accepts "b" anbnPDA 10
accepts "aa" anbnPDA 10
accepts "bb" anbnPDA 10
accepts "aab" anbnPDA 10
accepts "abb" anbnPDA 10

False

False

False

False

False

False

---
과제의 문법과 비슷한 중간단계로 연산자 우선순위는 확정했지만 왼쪽재귀가 있는 문법

```
term   ::= digit   |  '(' expr ')'
factor ::= term    |  factor '*' term
expr   ::= factor  |  expr   '+' factor
```

* $S\to EZ$
* $T\to a$
* $T\to ( ~ E ~ )$
* $F\to T$
* $F\to F ~ * ~ T$
* $E\to F$
* $E\to E ~ + ~ F$

<img src="https://graphviz.glitch.me/graphviz?layout=dot&format=svg&mode=download&graph=digraph%20G%20%7B%0Anode%20%5Bshape%3Dcircle%5D%3B%0A%0Astart%20-%3E%200%3B%0A0%20-%3E%200%20%5Blabel%3D%22%20%CE%B5%3B%20Z%2FEZ%20%20%5Cn%20%CE%B5%3B%20E%2FE%2BF%22%5D%3B%0A0%20-%3E%201%20%5Blabel%3D%22%20%CE%B5%3B%20E%2FF%22%5D%3B%0A1%20-%3E%201%20%5Blabel%3D%22%20%CE%B5%3B%20F%2FF*T%22%5D%3B%0A1%20-%3E%202%20%5Blabel%3D%22%20%CE%B5%3B%20F%2FT%22%5D%3B%0A2%20-%3E%202%20%5Blabel%3D%22%20%CE%B5%3B%20T%2Fa%20%20%20%20%20%20a%3B%20a%2F%CE%B5%20%20%20%20%5Cn%20%CE%B5%3B%20T%2F(E)%20%20%20)%3B%20)%2F%CE%B5%20%20%20%20%5Cn%20%20%20%20%20%20%20%20%20%20%20%20*%3B%20*%2F%CE%B5%22%5D%3B%0A2%20-%3E%200%20%5Blabel%3D%22%20(%3B%20(%2F%CE%B5%22%5D%3B%0A2%20-%3E%201%20%5Blabel%3D%22%20%2B%3B%20%2B%2F%CE%B5%22%5D%3B%0A2%20-%3E%203%20%5Blabel%3D%22%20%CE%B5%3B%20Z%2F%CE%B5%22%5D%3B%0A%0Astart%20%5Bshape%3Dnone%5D%3B%0A3%20%5Bshape%3Ddoublecircle%5D%3B%0A%7D" />

In [47]:
exprPDA :: PDA
exprPDA = (3,d,0,[3])
  where
    d = [ (0, e,__Z, [ _E,  Z],         0) -- start
        , (0, e,__E, [ _F],             1) -- produce E -> F
        , (0, e,__E, [ _E,_t,_F],       0) -- produce E -> E + F
        , (1, e,__F, [ _T],             2) -- produce F -> T
        , (1, e,__F, [ _F,_x,_T],       1) -- produce F -> F x T
        , (2, e,__T, [ _a],             2) -- produce T -> a
        , (2, e,__T, [_lp,_E,_rp],      2) -- producE T -> ( E )
        
        , (2, a,__a, [], 2) -- match a
        , (2,rp,__rp,[], 2) -- match )
        , (2, x,__x, [], 2) -- match *
        , (2,lp,__lp,[], 0) -- match (
        , (2, t,__t, [], 1) -- match +
        
        , (2, e,__Z, [], 3) -- finish
        ]
    e = Nothing
    a  = Just 'a'
    t  = Just '+'
    x  = Just '*'
    lp = Just '('
    rp = Just ')'
    __Z = Just Z
    __T = Just _T
    __F = Just _F
    __E = Just _E
    __a = Just _a
    __t = Just _t
    __x = Just _x
    __lp = Just _lp
    __rp = Just _rp

_T = N "T"
_F = N "F"
_E = N "E"
_a  = T 'a'
_t  = T '+'
_x  = T '*'
_lp  = T '('
_rp  = T ')'

In [48]:
(0,"(a+a)*a",                    [ Z]) |- (0,"(a+a)*a",                 [_E, Z]) $ exprPDA
(0,"(a+a)*a",                 [_E, Z]) |- (1,"(a+a)*a",                 [_F, Z]) $ exprPDA
(1,"(a+a)*a",                 [_F, Z]) |- (1,"(a+a)*a",           [_F,_x,_T, Z]) $ exprPDA
(1,"(a+a)*a",           [_F,_x,_T, Z]) |- (2,"(a+a)*a",           [_T,_x,_T, Z]) $ exprPDA
(1,"(a+a)*a",           [_F,_x,_T, Z]) |- (2,"(a+a)*a",           [_T,_x,_T, Z]) $ exprPDA
(2,"(a+a)*a",           [_T,_x,_T, Z]) |- (2,"(a+a)*a",   [_lp,_E,_rp,_x,_T, Z]) $ exprPDA
(2,"(a+a)*a",   [_lp,_E,_rp,_x,_T, Z]) |- (0, "a+a)*a",       [_E,_rp,_x,_T, Z]) $ exprPDA
(0, "a+a)*a",       [_E,_rp,_x,_T, Z]) |- (0, "a+a)*a", [_E,_t,_F,_rp,_x,_T, Z]) $ exprPDA
(0, "a+a)*a", [_E,_t,_F,_rp,_x,_T, Z]) |- (1, "a+a)*a", [_F,_t,_F,_rp,_x,_T, Z]) $ exprPDA
(1, "a+a)*a", [_F,_t,_F,_rp,_x,_T, Z]) |- (2, "a+a)*a", [_T,_t,_F,_rp,_x,_T, Z]) $ exprPDA
(2, "a+a)*a", [_T,_t,_F,_rp,_x,_T, Z]) |- (2, "a+a)*a", [_a,_t,_F,_rp,_x,_T, Z]) $ exprPDA
(2, "a+a)*a", [_a,_t,_F,_rp,_x,_T, Z]) |- (2,  "+a)*a",    [_t,_F,_rp,_x,_T, Z]) $ exprPDA
(2,  "+a)*a",    [_t,_F,_rp,_x,_T, Z]) |- (1,   "a)*a",       [_F,_rp,_x,_T, Z]) $ exprPDA
(1,   "a)*a",       [_F,_rp,_x,_T, Z]) |- (2,   "a)*a",       [_T,_rp,_x,_T, Z]) $ exprPDA
(2,   "a)*a",       [_T,_rp,_x,_T, Z]) |- (2,   "a)*a",       [_a,_rp,_x,_T, Z]) $ exprPDA
(2,   "a)*a",       [_a,_rp,_x,_T, Z]) |- (2,    ")*a",          [_rp,_x,_T, Z]) $ exprPDA
(2,    ")*a",          [_rp,_x,_T, Z]) |- (2,     "*a",              [_x,_T, Z]) $ exprPDA
(2,     "*a",              [_x,_T, Z]) |- (2,      "a",                 [_T, Z]) $ exprPDA
(2,      "a",                 [_T, Z]) |- (2,      "a",                 [_a, Z]) $ exprPDA
(2,      "a",                 [_a, Z]) |- (2,       "",                     [Z]) $ exprPDA
(2,       "",                     [Z]) |- (3,       "",                      []) $ exprPDA

True

True

True

True

True

...

True

True

True

True

True